In [1]:
import scipy as sp
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib
import squarify
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
#Term frequency vectorizer
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.linear_model import Ridge, LogisticRegression
import re
import xgboost
import scipy



c:\users\kj.park\appdata\local\programs\python\python36\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
data_train = pd.read_csv("train.tsv", sep='\t', header=0)
data_train.head()

,train_id,name,item_condition_id,category_name,brand_name,price,shipping,item_description
0,0,MLB Cincinnati Reds T Shirt Size XL,3,Men/Tops/T-shirts,NaN,10.0,1,No description yet
1,1,Razer BlackWidow Chroma Keyboard,3,Electronics/Computers & Tablets/Components & P...,Razer,52.0,0,This keyboard is in great condition and works ...
2,2,AVA-VIV Blouse,1,Women/Tops & Blouses/Blouse,Target,10.0,1,Adorable top with a hint of lace and a key hol...
3,3,Leather Horse Statues,1,Home/Home Décor/Home Décor Accents,NaN,35.0,1,New with tags. Leather horses. Retail for [rm]...
4,4,24K GOLD plated rose,1,Women/Jewelry/Necklaces,NaN,44.0,0,Complete with certificate of authenticity


In [3]:
data_test = pd.read_csv("test.tsv", sep='\t', header=0)
data_test.head()

,test_id,name,item_condition_id,category_name,brand_name,shipping,item_description
0,0,"Breast cancer ""I fight like a girl"" ring",1,Women/Jewelry/Rings,NaN,1,Size 7
1,1,"25 pcs NEW 7.5""x12"" Kraft Bubble Mailers",1,Other/Office supplies/Shipping Supplies,NaN,1,"25 pcs NEW 7.5""x12"" Kraft Bubble Mailers Lined..."
2,2,Coach bag,1,Vintage & Collectibles/Bags and Purses/Handbag,Coach,1,Brand new coach bag. Bought for [rm] at a Coac...
3,3,Floral Kimono,2,Women/Sweaters/Cardigan,NaN,0,-floral kimono -never worn -lightweight and pe...
4,4,Life after Death,3,Other/Books/Religion & Spirituality,NaN,1,Rediscovering life after the loss of a loved o...


In [4]:
data = pd.concat([data_train, data_test], 0)
nrow_train = data_train.shape[0]
print(nrow_train)
y  = np.log1p(data_train["price"])
y_train = np.log1p(data_train["price"])

1482535


In [5]:
non_text = [',','-','.','&','%','#', '(',')','[',']','*','!','✨','▪️','"','~',"'"]
for symbols in non_text: 
    data['category_name'] = data['category_name'].str.replace(symbols, '') 
    data['item_description'] = data['item_description'].str.replace(symbols, '') 
    data['name'] = data['name'].str.replace(symbols, '')
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2175894 entries, 0 to 693358
Data columns (total 9 columns):
brand_name           object
category_name        object
item_condition_id    int64
item_description     object
name                 object
price                float64
shipping             int64
test_id              float64
train_id             float64
dtypes: float64(3), int64(2), object(4)
memory usage: 166.0+ MB


In [6]:
data = data.apply(lambda x: x.astype(str).str.lower())

In [7]:
data['price'] = data['price'].astype('float')

In [8]:
data['primary_cat'] = data.category_name.str.extract('([^/]+)/[^/]+/[^/]+')
data['secondary_cat'] = data.category_name.str.extract('[^/]+/([^/]+)/[^/]+')
data['tertiary_cat'] = data.category_name.str.extract('[^/]+/[^/]+/([^/]+)')


c:\users\kj.park\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  """Entry point for launching an IPython kernel.
c:\users\kj.park\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  
c:\users\kj.park\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  This is separate from the ipykernel package so we can avoid doing imports until


In [9]:
data['brand_name'] = data['brand_name'].fillna(value = 'none', inplace=False, axis=0)
data['item_description'] = data['item_description'].fillna(value='none', inplace=False, axis=0)
data['primary_cat'] = data['primary_cat'].fillna(value='none', inplace=False, axis=0)
data['secondary_cat'] = data['secondary_cat'].fillna(value='none', inplace=False, axis=0)
data['tertiary_cat'] = data['tertiary_cat'].fillna(value='none', inplace=False, axis=0)
data['name'] = data['name'].fillna(value = 'none', inplace=False, axis=0)


In [10]:
print(data['brand_name'].isnull().sum())
print(data['item_description'].isnull().sum())
print(data['primary_cat'].isnull().sum())
print(data['secondary_cat'].isnull().sum())
print(data['tertiary_cat'].isnull().sum())
print(data['name'].isnull().sum())


0
0
0
0
0
0


In [11]:
count = CountVectorizer(stop_words='english')
vect = LabelBinarizer(sparse_output=True)
X_name = count.fit_transform(data['name'])
X_cat_1 = count.fit_transform(data['primary_cat'])
X_cat_2 = count.fit_transform(data['secondary_cat'])
X_cat_3 = count.fit_transform(data['tertiary_cat'])
X_brand = vect.fit_transform(data['brand_name'])
count_desc = TfidfVectorizer(max_features = 50000, ngram_range = (1,3),stop_words = "english")
X_desc = count_desc.fit_transform(data["item_description"])
X_dummies = scipy.sparse.csr_matrix(pd.get_dummies(data[[
    "item_condition_id", "shipping"]], sparse = True).values)

In [12]:
X = scipy.sparse.hstack((X_dummies, 
                         X_desc,
                         X_brand,
                         X_cat_1,
                         X_cat_2,
                         X_cat_3,
                         X_name)).tocsr()

In [13]:
X_train = X[:nrow_train]
X_test = X[nrow_train:]

In [14]:
train_X, valid_X, train_y, valid_y = train_test_split(X_train, y_train, test_size = 0.3, random_state = 42) 

In [15]:
train = xgboost.DMatrix(train_X, label=train_y)
valid = xgboost.DMatrix(valid_X, label=valid_y)
Xtest = xgboost.DMatrix(X_test)
watchlist = [(train, 'train'), (valid, 'valid')]

for i in train_item_d.values:
    x = tokenizer(str(i))

In [16]:
params = {'min_child_weight': 25, 'eta': 0.013, 'colsample_bytree': 0.60, 'max_depth': 18,
            'subsample': 0.80, 'lambda': 2.08, 'nthread': -1, 'booster' : 'gbtree', 'silent': 1,
            'eval_metric': 'rmse', 'objective': 'reg:linear'}

gbm = xgboost.train(params, train, 500, watchlist, early_stopping_rounds=25, verbose_eval=20)

[0]	train-rmse:2.55747	valid-rmse:2.55969
Multiple eval metrics have been passed: 'valid-rmse' will be used for early stopping.

Will train until valid-rmse hasn't improved in 25 rounds.
[20]	train-rmse:2.00805	valid-rmse:2.01032
[40]	train-rmse:1.5927	valid-rmse:1.5952
[60]	train-rmse:1.28218	valid-rmse:1.28504
[80]	train-rmse:1.05364	valid-rmse:1.05706
[100]	train-rmse:0.888411	valid-rmse:0.89256
[120]	train-rmse:0.771405	valid-rmse:0.776471
[140]	train-rmse:0.690502	valid-rmse:0.69651
[160]	train-rmse:0.63575	valid-rmse:0.642682
[180]	train-rmse:0.598893	valid-rmse:0.606711
[200]	train-rmse:0.574479	valid-rmse:0.583019
[220]	train-rmse:0.558068	valid-rmse:0.567156
[240]	train-rmse:0.546772	valid-rmse:0.556319
[260]	train-rmse:0.538832	valid-rmse:0.548779
[280]	train-rmse:0.533235	valid-rmse:0.543513
[300]	train-rmse:0.528827	valid-rmse:0.539441
[320]	train-rmse:0.525488	valid-rmse:0.536371
[340]	train-rmse:0.522646	valid-rmse:0.533793
[360]	train-rmse:0.520258	valid-rmse:0.531616
[3

In [18]:
preds = gbm.predict(Xtest)

In [20]:
data_test["price"] = np.expm1(preds)
data_test[["test_id", "price"]].to_csv("kyungjunism_submission_xgb.csv", index = False)